# Preprocessor
   1. Convert TIFF video to individual TIFF images
        > Clearing the folder and index+1 to have the range 1-300 added
   2. Implement OpenCV for circle detection 
   3. Identify the Largest circle detected
   4. Establish a bounding rectangular area around the circle
   5. Perform cropping based on the bounding rectangular
   6. Save processed data

#### 1. Converting TIFF

In [14]:
import os
import numpy as np
import tifffile
from PIL import Image

# Read the image
imageAll = tifffile.imread('CaSki17.tif')
lengthOfStack = imageAll.shape[0]  # Get the length of the stack
dirToStore = "C:/Users/mkana/Desktop/GPMV/GPMV_new/SingleTiffs/"

for file in os.listdir(dirToStore):
    os.remove(os.path.join(dirToStore, file))

for index in range(lengthOfStack):  # Loop through each frame
    image = np.array(imageAll[index, :, :], dtype=np.uint16)
    image = Image.fromarray(image) # Convert to PIL Image
    image.save(f"{dirToStore}{index+1}.tif")

<tifffile.TiffTag 5033 @530880> coercing invalid ASCII to bytes


#### 2. Implementing OpenCV 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from skimage import io, color, filters, measure

image_path = 'C:/Users/mkana/Desktop/GPMV/GPMV_new/SingleTiffs/1.tif'

try:
    image = io.imread(image_path)
except FileNotFoundError:
    print("Error: Unable to load the image.")
    exit()

# Convert image to grayscale
if image.ndim > 2:
    image_gray = color.rgb2gray(image)
else:
    image_gray = image

# Apply median blur for noise reduction (can be also Gaussian blur)
blurred = filters.median(image_gray)

# Threshold the image to obtain binary image
thresholded = blurred > filters.threshold_otsu(blurred)

# Label connected components
label_image = measure.label(thresholded)

# Create a black image for drawing contours
contour_image = np.zeros_like(image, dtype=np.uint8)

# Iterate through labeled regions
for region in measure.regionprops(label_image):
    # Get the properties of the region
    min_row, min_col, max_row, max_col = region.bbox
    centroid_row, centroid_col = region.centroid
    radius = int(max((max_row - min_row), (max_col - min_col)) / 2)

    # Draw the circle on the contour image
    cv2.circle(contour_image, (int(centroid_col), int(centroid_row)), radius, (255, 255, 255), 2)

# Save the contour image
contour_image_path = os.path.splitext(image_path)[0] + '_contour.tif'
io.imsave(contour_image_path, contour_image)

# Display the contour image
plt.imshow(contour_image)
plt.title('Detected Circular Structures')
plt.axis('off')
plt.show()


ModuleNotFoundError: No module named 'skimage'